In [1]:
import cv2

import glob

import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import pandas as pd

import pydicom as dicom
import pylibjpeg

In [2]:
# dataset root path
root = '/media/luisa/Volume/AML/siim-covid19-detection'

In [3]:
# 1) blacklist duplicates of images. If one is annotated, keep that. If none are anotated, keep only one

In [4]:

duplicates_images = pd.read_csv("/media/luisa/Volume/AML/duplicates/duplicates.csv", header = None, names = [0, 1, 2, 3, 4, 5, 6, 7, 8 ])

duplicates_study = pd.read_csv("/media/luisa/Volume/AML/duplicates/duplicates_study.csv")

In [5]:
annotations_images = pd.read_csv("/media/luisa/Volume/AML/siim-covid19-detection/train_image_level.csv")
annotations_study = pd.read_csv("/media/luisa/Volume/AML/siim-covid19-detection/train_study_level.csv")

In [6]:
annotations_images.head()

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


In [7]:
duplicates_images.head()

,0,1,2,3,4,5,6,7,8
0,6534a837497d_image,74077a8e3b7c_image,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9e4824fcee2e_image,ea2117b53323_image,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19fc87ff0612_image,eea3a910fa9e_image,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1ea01196514a_image,961e67cadcdc_image,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,70fbcd6dcc53_image,9b1de1c45491_image,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(annotations_images.loc[annotations_images["id"] == duplicates_images[0][1]]['boxes'].notna().any())

True


In [9]:
# Dataframe with annotations corresponding to duplicate_images

duplicates_annotations = np.zeros(duplicates_images.shape, dtype = object)


for i in range (0, 8):
    for k in range( len(duplicates_images[i])) :
        #duplicates_annotations[i][k] = duplicates_images[i][k]
        if duplicates_images.notna()[i][k] == True:
            if annotations_images.loc[annotations_images["id"] == duplicates_images[i][k]]['boxes'].notna().any() == True:
                duplicates_annotations[k][i] = annotations_images.loc[annotations_images["id"] == duplicates_images[i][k]]['boxes'].tolist()


        
duplicates_annotations = pd.DataFrame(duplicates_annotations)
duplicates_annotations.head()

,0,1,2,3,4,5,6,7,8
0,0,"[[{'x': 2175.24285, 'y': 1123.72368, 'width': ...",0,0,0,0,0,0,0
1,"[[{'x': 817.77961, 'y': 1075.34501, 'width': 6...",0,0,0,0,0,0,0,0
2,"[[{'x': 2039.52277, 'y': 1058.05502, 'width': ...",0,0,0,0,0,0,0,0
3,0,"[[{'x': 63.92032, 'y': 655.62553, 'width': 642...",0,0,0,0,0,0,0
4,"[[{'x': 421.44957, 'y': 437.19629, 'width': 66...",0,0,0,0,0,0,0,0


In [10]:
duplicates_annotations.to_csv('duplicate_annotations.csv')

In [11]:
#if there is ONE annotation, use the image with the annotation. if there is NO annotation in the row, use the first in the row. if there is MORE than one annotation, blacklist all in row
has_annotation = np.where(duplicates_annotations !=0, 1, 0)

blacklist = []


for i in range (0, 9):
    for k in range( len(duplicates_images[i])) :
        if duplicates_images.notna()[i][k] == True:
          

            #blacklist all images if more than one annotation exists for the duplicates
            if np.sum(has_annotation[k]) >1:
                blacklist.append(duplicates_images[i][k])
                #print(duplicates_images[i][k])
                
            #blacklist image if it has no annotation, but annotation exists for a duplicate
            elif np.sum(has_annotation[k]) == 1 and has_annotation[k][i] == 0 :
                blacklist.append(duplicates_images[i][k])
                #print(duplicates_images[i][k])
                
            #if all duplicates without annotation, only use first image in row of duplicates
            elif np.sum(has_annotation[k]) == 0 and i >0:
                blacklist.append(duplicates_images[i][k])
                #print(duplicates_images[i][k])
                
                
                
print("number of images on blacklist:")                
print(len(blacklist))


number of images on blacklist:
108


In [12]:
# 2) If a study has more than one image, but only one has annotations, blackist the ones without

In [13]:
annotations_images['has_annotation'] = annotations_images['boxes'].notna()
annotations_images['only one image in study'] = ~annotations_images['StudyInstanceUID'].duplicated(keep=False)



In [14]:
annotations_not_unique_study =  annotations_images[annotations_images['only one image in study']== False]

In [15]:
print(annotations_not_unique_study.shape)

(512, 6)


In [16]:
studies_with_mult_images = np.unique(annotations_not_unique_study['StudyInstanceUID'])
print(len(studies_with_mult_images))

232


In [17]:
annotations_not_unique_study.head()

,id,boxes,label,StudyInstanceUID,has_annotation,only one image in study
22,00c1515729a8_image,NaN,none 1 0 0 1 1,1a58b43cf286,False,False
24,00e3a7e91a34_image,NaN,none 1 0 0 1 1,74ba8f2badcb,False,False
27,0102b5cac730_image,NaN,none 1 0 0 1 1,81f6b4b343f7,False,False
123,04a0b90d7875_image,NaN,none 1 0 0 1 1,effcdb73627e,False,False
130,04cc2f7f4c4b_image,NaN,none 1 0 0 1 1,0fd2db233deb,False,False


In [18]:
annotations_not_unique_study['study_has_annotation'] = [annotations_not_unique_study[annotations_not_unique_study['StudyInstanceUID'] == i]['has_annotation'].sum() for i in annotations_not_unique_study['StudyInstanceUID']]

/home/luisa/anaconda3/envs/torch/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [19]:
annotations_not_unique_study.head()

,id,boxes,label,StudyInstanceUID,has_annotation,only one image in study,study_has_annotation
22,00c1515729a8_image,NaN,none 1 0 0 1 1,1a58b43cf286,False,False,1
24,00e3a7e91a34_image,NaN,none 1 0 0 1 1,74ba8f2badcb,False,False,0
27,0102b5cac730_image,NaN,none 1 0 0 1 1,81f6b4b343f7,False,False,1
123,04a0b90d7875_image,NaN,none 1 0 0 1 1,effcdb73627e,False,False,1
130,04cc2f7f4c4b_image,NaN,none 1 0 0 1 1,0fd2db233deb,False,False,1


In [20]:
for i in range(len(annotations_not_unique_study)):
    id_ = annotations_not_unique_study['id'].iloc[i]
    
    #blacklist image id if it has no annotation, but study has annotation
    if (annotations_not_unique_study['has_annotation'].iloc[i] == False) and (annotations_not_unique_study['study_has_annotation'].iloc[i] > 0):
        blacklist.append(id_)

blacklist = np.unique(blacklist)

print("number of images on blacklist:")                
print(len(blacklist))


number of images on blacklist:
248


In [21]:
#pre-process train set by removing all images on blacklist
print(len(annotations_images))

annotations_images_clean = annotations_images[~annotations_images["id"].isin(blacklist)]
print(len(annotations_images_clean))
print(len(annotations_images) - len(annotations_images_clean))
print(len(blacklist))



6334
6086
248
248


In [22]:
# set a string study_label in study level annotations
annotations_study['study_label'] = np.where(annotations_study['Negative for Pneumonia'] == 1, 'Negative for Pneumonia', 0)
annotations_study['study_label'] = np.where(annotations_study['Typical Appearance'] == 1, 'Typical Appearance', annotations_study['study_label'])
annotations_study['study_label'] = np.where(annotations_study['Indeterminate Appearance'] == 1, 'Indeterminate Appearance', annotations_study['study_label'])
annotations_study['study_label'] = np.where(annotations_study['Atypical Appearance'] == 1, 'Atypical Appearance', annotations_study['study_label'])

annotations_study['StudyInstanceUID'] = annotations_study['id'].str.split('_').str[0]

annotations_study = annotations_study.drop(columns = ['id','Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance' ])
annotations_study.head()


,study_label,StudyInstanceUID
0,Typical Appearance,00086460a852
1,Atypical Appearance,000c9c05fd14
2,Negative for Pneumonia,00292f8c37bd
3,Negative for Pneumonia,005057b3f880
4,Atypical Appearance,0051d9b12e72


In [23]:
annotations_images_clean = annotations_images_clean.merge(annotations_study, how = 'left', on = 'StudyInstanceUID')
annotations_images_clean.head()

,id,boxes,label,StudyInstanceUID,has_annotation,only one image in study,study_label
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,True,True,Typical Appearance
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,False,True,Negative for Pneumonia
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,True,True,Typical Appearance
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,True,True,Atypical Appearance
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,True,True,Typical Appearance


In [24]:
print(annotations_images_clean.shape)

(6086, 7)


In [25]:
annotations_images_clean.to_csv("train_image_level_clean.csv")

In [26]:
# 3) add path to image to dataframe

In [27]:
train_df = annotations_images_clean.copy()

In [28]:
train_studies = train_df['StudyInstanceUID']
print(len(train_studies))
print(len(np.unique(train_studies)))

6086
6052


In [29]:
train_files = sorted(glob.glob(os.path.join(root, 'train/*/*/*.dcm')))
print(f'Number of training files: {len(train_files)}')

Number of training files: 6334


In [30]:
print(train_files[:10])

['/media/luisa/Volume/AML/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm', '/media/luisa/Volume/AML/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm', '/media/luisa/Volume/AML/siim-covid19-detection/train/00292f8c37bd/73120b4a13cb/f6293b1c49e2.dcm', '/media/luisa/Volume/AML/siim-covid19-detection/train/005057b3f880/e34afce999c5/3019399c31f4.dcm', '/media/luisa/Volume/AML/siim-covid19-detection/train/0051d9b12e72/152f6ec68d86/bb4b1da810f3.dcm', '/media/luisa/Volume/AML/siim-covid19-detection/train/00792b5c8852/1f52bcb3143e/3fadf4b48db3.dcm', '/media/luisa/Volume/AML/siim-covid19-detection/train/00908ffd2d08/e1bb4145673c/bf1f75117093.dcm', '/media/luisa/Volume/AML/siim-covid19-detection/train/009bc005edaa/8713301456d4/1df3e98f79be.dcm', '/media/luisa/Volume/AML/siim-covid19-detection/train/00a76543ed93/4a223cccbe04/ad8d4a5ba8f0.dcm', '/media/luisa/Volume/AML/siim-covid19-detection/train/00a87235ca36/b7a93187765f/09cf9767a7bf.dcm']


In [31]:
files_df = pd.DataFrame(train_files, columns=['path'])

In [32]:
# paths: root/train/[study]/[some folder]/[image].dcm

files_df['image'] = files_df['path'].str.split('/', expand = False).str[9]
files_df['study'] = files_df['path'].str.split('/', expand = False).str[7]

files_df['id'] = files_df['image'].str.split('.', expand = False).str[0] + '_image'
files_df.head()


,path,image,study,id
0,/media/luisa/Volume/AML/siim-covid19-detection...,65761e66de9f.dcm,00086460a852,65761e66de9f_image
1,/media/luisa/Volume/AML/siim-covid19-detection...,51759b5579bc.dcm,000c9c05fd14,51759b5579bc_image
2,/media/luisa/Volume/AML/siim-covid19-detection...,f6293b1c49e2.dcm,00292f8c37bd,f6293b1c49e2_image
3,/media/luisa/Volume/AML/siim-covid19-detection...,3019399c31f4.dcm,005057b3f880,3019399c31f4_image
4,/media/luisa/Volume/AML/siim-covid19-detection...,bb4b1da810f3.dcm,0051d9b12e72,bb4b1da810f3_image


In [33]:
print(train_df.shape)
print(files_df.shape)

(6086, 7)
(6334, 4)


In [34]:
train_df.head()

,id,boxes,label,StudyInstanceUID,has_annotation,only one image in study,study_label
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,True,True,Typical Appearance
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,False,True,Negative for Pneumonia
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,True,True,Typical Appearance
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,True,True,Atypical Appearance
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,True,True,Typical Appearance


In [35]:
train_df = train_df.drop(columns = ['has_annotation', 'only one image in study'])
train_df = train_df.merge(files_df, how = 'left', on = 'id')
train_df['check'] = np.where(train_df['StudyInstanceUID'] == train_df['study'], True, False)
train_df.head()

,id,boxes,label,StudyInstanceUID,study_label,path,image,study,check
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,Typical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,000a312787f2.dcm,5776db0cec75,True
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,Negative for Pneumonia,/media/luisa/Volume/AML/siim-covid19-detection...,000c3a3f293f.dcm,ff0879eb20ed,True
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,Typical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,0012ff7358bc.dcm,9d514ce429a7,True
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,Atypical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,001398f4ff4f.dcm,28dddc8559b2,True
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,Typical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,001bd15d1891.dcm,dfd9fdd85a3e,True


In [36]:
print(np.where(train_df['check'] == False))

(array([], dtype=int64),)


In [37]:
train_df = train_df.drop(columns = ['study', 'check'])
train_df.head()

,id,boxes,label,StudyInstanceUID,study_label,path,image
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,Typical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,000a312787f2.dcm
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,Negative for Pneumonia,/media/luisa/Volume/AML/siim-covid19-detection...,000c3a3f293f.dcm
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,Typical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,0012ff7358bc.dcm
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,Atypical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,001398f4ff4f.dcm
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,Typical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,001bd15d1891.dcm


In [39]:
# add col with image sizes

In [43]:
train_df['image_size'] = [dicom.dcmread(i).pixel_array.shape for i in train_df['path']] 

/home/luisa/anaconda3/envs/torch/lib/python3.9/site-packages/pydicom/pixel_data_handlers/numpy_handler.py:359: UserWarning: The length of the pixel data in the dataset (13262360 bytes) indicates it contains excess padding. 216296 bytes will be removed from the end of the data
  warnings.warn(msg)


In [44]:
train_df.head()

,id,boxes,label,StudyInstanceUID,study_label,path,image,image_size
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,Typical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,000a312787f2.dcm,"(3488, 4256)"
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed,Negative for Pneumonia,/media/luisa/Volume/AML/siim-covid19-detection...,000c3a3f293f.dcm,"(2320, 2832)"
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,Typical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,0012ff7358bc.dcm,"(2544, 3056)"
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,Atypical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,001398f4ff4f.dcm,"(3520, 4280)"
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,Typical Appearance,/media/luisa/Volume/AML/siim-covid19-detection...,001bd15d1891.dcm,"(2800, 3408)"


In [45]:
train_df.to_csv('train_image_level_clean_paths.csv')
train_df.to_csv('/media/luisa/Volume/AML/train_image_level_clean_paths.csv')